1)	Peguem uma empresa que respondeu menos e façam o seguinte:
-	Filtrar as reclamações que a empresa não respondeu
-	A IA irá traduzir as reclamações para Português
-	A IA vai fazer um resumo da reclamação
-	A IA vai detectar o nível de risco do cliente (BAIXO, MEDIO, ALTO)
Risco Alto: Palavras que ameaça diretamente a empresa com processo, advogado, linguagem agressiva ou tóxica. Ameaça deixar de utilizar os produtos da empresa.
Risco Médio: Cliente não ameaça diretamente a empresa mas utiliza algumas palavras agressivas demonstrando sinal de estresse mas não dá indícios de deixar de consumir algum produto da empresa.
Risco Baixo: Cliente faz a reclamação sem ameaças a empresa e demonstra insatisfação de forma educada.
-	A IA vai gerar um plano de ação

Ordenar o plano de ação por classificação de risco

Saida Tabela:
ID da reclamação:
Data da reclamação:
Empresa:
Resumo da Reclamação:
Risco:
Plano de ação:


## Pré-processamento da base

In [ ]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('./data/consumer_complaints.db')
cursor = conn.cursor()

# Imprimindo nome das colunas
cursor.execute(f"PRAGMA table_info(consumer_complaints);")
colunas = cursor.fetchall()
nomes_colunas = tuple(coluna[1] for coluna in colunas)
print(nomes_colunas)

# Visualizando algumas linhas da coluna
cursor.execute("SELECT * FROM consumer_complaints;")
for row in cursor.fetchall()[:10]:
    print(row)



('date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company_public_response', 'company', 'state', 'zipcode', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response_to_consumer', 'timely_response', 'consumer_disputed?', 'complaint_id')
('08/30/2013', 'Mortgage', 'Other mortgage', 'Loan modification,collection,foreclosure', None, None, None, 'U.S. Bancorp', 'CA', '95993', None, None, 'Referral', '09/03/2013', 'Closed with explanation', 'Yes', 'Yes', 511074)
('08/30/2013', 'Mortgage', 'Other mortgage', 'Loan servicing, payments, escrow account', None, None, None, 'Wells Fargo & Company', 'CA', '91104', None, None, 'Referral', '09/03/2013', 'Closed with explanation', 'Yes', 'Yes', 511080)
('08/30/2013', 'Credit reporting', None, 'Incorrect information on credit report', 'Account status', None, None, 'Wells Fargo & Company', 'NY', '11764', None, None, 'Postal mail', '09/18/2013', 'Closed with explanation

Quantas reclamações que tem descrição da reclamação do cliente tem por empresa?  
##### 1 - Só consigo trabalhar com as reclamações que tem consentimento do cliente pra descrever a narrativa

In [3]:
pd_reclamacoes_totais = pd.read_sql_query("""select * from consumer_complaints where consumer_consent_provided like 'Consent provided'""", conn)
display(pd_reclamacoes_totais[:10])

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,03/19/2015,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,XXXX has claimed I owe them {$27.00} for XXXX ...,None,"Diversified Consultants, Inc.",NY,121XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290516
1,03/19/2015,Consumer Loan,Vehicle loan,Managing the loan or lease,None,Due to inconsistencies in the amount owed that...,None,M&T Bank Corporation,VA,221XX,Servicemember,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290492
2,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",None,In XX/XX/XXXX my wages that I earned at my job...,None,Wells Fargo & Company,CA,946XX,None,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1290524
3,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",None,I have an open and current mortgage with Chase...,None,JPMorgan Chase & Co.,CA,900XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1290253
4,03/19/2015,Mortgage,Conventional fixed mortgage,Credit decision / Underwriting,None,XXXX was submitted XX/XX/XXXX. At the time I s...,None,Rushmore Loan Management Services LLC,CA,956XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1292137
5,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",None,Experian is reporting my OPEN and CURRENT Mort...,Company chooses not to provide a public response,Experian,CA,900XX,Older American,Consent provided,Web,03/25/2015,Closed with explanation,Yes,No,1290254
6,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",None,This complaint is against Wells Fargo Bank for...,None,Wells Fargo & Company,CA,931XX,None,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290532
7,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",None,I spoke to XXXX of green tree representatives ...,None,Ditech Financial LLC,CA,948XX,None,Consent provided,Web,03/19/2015,Closed with non-monetary relief,Yes,No,1290534
8,03/19/2015,Credit card,None,Other,None,i opened XXXX Bank of America credit cards 15-...,Company chooses not to provide a public response,Bank of America,OR,971XX,None,Consent provided,Web,03/19/2015,Closed with monetary relief,Yes,Yes,1292209
9,03/19/2015,Consumer Loan,Vehicle loan,Taking out the loan or lease,None,I applied for a loan with XXXX XXXX and had pu...,None,Hyundai Capital America,MS,392XX,None,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1292159


Vamos ver quantas reclamações temos para trabalhar

In [4]:
pd_reclamacoes_totais.shape

(66807, 18)

Temos 18 colunas e 66807 reclamações. Destas reclamações, vamos agrupar a quantidade por empresas

In [5]:
cursor.execute("""select company, count(*)
                from consumer_complaints
                where consumer_complaint_narrative is not NULL
                group by company
                order by 2 desc;""")

pd_reclamacoes = pd_reclamacoes_totais[['company', 'complaint_id']].groupby('company').count().sort_values(by='complaint_id', ascending=False)
display(pd_reclamacoes[:10])

print(f'Total de reclamações: {pd_reclamacoes['complaint_id'].sum()}')

,complaint_id
company,
Equifax,4195
Experian,3934
"TransUnion Intermediate Holdings, Inc.",3863
Bank of America,3498
Wells Fargo & Company,3075
Citibank,2789
JPMorgan Chase & Co.,2586
Ocwen,1620
Capital One,1504


Total de reclamações: 66807


Renomearei a coluna 'complaint_id' para 'qtd_reclamacoes'

In [6]:
pd_reclamacoes.rename(columns={'complaint_id': 'qtd_reclamacoes'}, inplace=True)
display(pd_reclamacoes[:10])

,qtd_reclamacoes
company,
Equifax,4195
Experian,3934
"TransUnion Intermediate Holdings, Inc.",3863
Bank of America,3498
Wells Fargo & Company,3075
Citibank,2789
JPMorgan Chase & Co.,2586
Ocwen,1620
Capital One,1504


Quantas reclamações que o usuário descreveu a reclamação tiveram resposta por companhia?

In [7]:
pd_reclamacoes_respondidas = pd.read_sql_query("""select company, count(company_public_response) as reclamacoes_respondidas
                                                from consumer_complaints
                                                where consumer_complaint_narrative is not NULL
                                                and company_public_response is not NULL
                                                group by company
                                                order by 2 desc""", conn)
display(pd_reclamacoes_respondidas.head(10))

,company,reclamacoes_respondidas
0,Experian,3934
1,"TransUnion Intermediate Holdings, Inc.",3605
2,Bank of America,3493
3,Wells Fargo & Company,2740
4,Citibank,2524
5,U.S. Bancorp,1077
6,Ditech Financial LLC,785
7,ERC,513
8,"Select Portfolio Servicing, Inc",476
9,"SunTrust Banks, Inc.",425


Preciso agora computar a taxa de reclamações respondidas = reclamações respondidas / total de reclamações com reclamação dos clientes  
Irei juntar as duas tabelas para ter essas informações para fazer o cálculo
Tem que ser **LEFT JOIN** porque pode ter empresa SEM RECLAMAÇÕES RESPONDIDAS

In [8]:
pd_reclamacoes_new = pd.merge(pd_reclamacoes, pd_reclamacoes_respondidas, on='company', how='left')
display(pd_reclamacoes_new.head(10))

,company,qtd_reclamacoes,reclamacoes_respondidas
0,Equifax,4195,NaN
1,Experian,3934,3934.0
2,"TransUnion Intermediate Holdings, Inc.",3863,3605.0
3,Bank of America,3498,3493.0
4,Wells Fargo & Company,3075,2740.0
5,Citibank,2789,2524.0
6,JPMorgan Chase & Co.,2586,NaN
7,Ocwen,1620,20.0
8,Capital One,1504,NaN
9,Synchrony Financial,1375,123.0


Repara que ao fazer o merge, algumas reclamações de certas companhias vieram com valor NaN.  
Ou seja, pode ser que não tenha reclamações respondidas destas empresas.  
Precisamos tratar esses nulos para fazer as contas

In [9]:
# Vou ver se valida a hipótese acima. Vou tentar pesquisar na outra tabela a empresa Equifax
pd_reclamacoes_respondidas.query("company == 'Equifax'")

,company,reclamacoes_respondidas


Realmente a empresa Equifax não está na tabela de reclamações respondidas.

In [10]:
pd_reclamacoes_new.fillna(0, inplace=True)
display(pd_reclamacoes_new.head(10))

,company,qtd_reclamacoes,reclamacoes_respondidas
0,Equifax,4195,0.0
1,Experian,3934,3934.0
2,"TransUnion Intermediate Holdings, Inc.",3863,3605.0
3,Bank of America,3498,3493.0
4,Wells Fargo & Company,3075,2740.0
5,Citibank,2789,2524.0
6,JPMorgan Chase & Co.,2586,0.0
7,Ocwen,1620,20.0
8,Capital One,1504,0.0
9,Synchrony Financial,1375,123.0


Vou dar uma olhada no tipo das colunas. Essa nova coluna (reclamações respondidas) aparentemente não está no tipo correto (inteiro)  
Se não tiver, **irei fazer a conversão de tipo de função** 

In [11]:
pd_reclamacoes_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296 entries, 0 to 2295
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  2296 non-null   object 
 1   qtd_reclamacoes          2296 non-null   int64  
 2   reclamacoes_respondidas  2296 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 53.9+ KB


In [12]:
pd_reclamacoes_new['reclamacoes_respondidas'] = pd_reclamacoes_new['reclamacoes_respondidas'].astype(int)
display(pd_reclamacoes_new.head(10))

,company,qtd_reclamacoes,reclamacoes_respondidas
0,Equifax,4195,0
1,Experian,3934,3934
2,"TransUnion Intermediate Holdings, Inc.",3863,3605
3,Bank of America,3498,3493
4,Wells Fargo & Company,3075,2740
5,Citibank,2789,2524
6,JPMorgan Chase & Co.,2586,0
7,Ocwen,1620,20
8,Capital One,1504,0
9,Synchrony Financial,1375,123


Vamos computar a porcentagem e ordenar por ordem de maior taxa de resposta para menor taxa de resposta.

In [13]:
pd_reclamacoes_new['percentual_respondidas'] = round(pd_reclamacoes_new['reclamacoes_respondidas'] / pd_reclamacoes_new['qtd_reclamacoes'], 5)
display(pd_reclamacoes_new.sort_values(by=['percentual_respondidas', 'qtd_reclamacoes'], ascending=False).head(20))

,company,qtd_reclamacoes,reclamacoes_respondidas,percentual_respondidas
1,Experian,3934,3934,1.0
21,"Select Portfolio Servicing, Inc",476,476,1.0
31,Ally Financial Inc.,326,326,1.0
32,USAA Savings,319,319,1.0
42,Regions Financial Corporation,185,185,1.0
46,Navy FCU,167,167,1.0
48,Sallie Mae,161,161,1.0
60,Loan Care,139,139,1.0
70,Focus Holding Company,115,115,1.0
72,"First National Collection Bureau, Inc.",114,114,1.0


Quando tenho uma base muito pequena de reclamações como visto acima, ter um percentual de 100% de taxa de resposta não se torna muito relevante para o problema.  
Então eu vou focar em **empresas que tem mais do que 400 reclamações** para ter uma amostra considerável de taxa de resposta.  

In [14]:
pd_reclamacoes_top400 = pd_reclamacoes_new.query('qtd_reclamacoes > 400')
display(pd_reclamacoes_top400)

,company,qtd_reclamacoes,reclamacoes_respondidas,percentual_respondidas
0,Equifax,4195,0,0.00000
1,Experian,3934,3934,1.00000
2,"TransUnion Intermediate Holdings, Inc.",3863,3605,0.93321
3,Bank of America,3498,3493,0.99857
4,Wells Fargo & Company,3075,2740,0.89106
5,Citibank,2789,2524,0.90498
6,JPMorgan Chase & Co.,2586,0,0.00000
7,Ocwen,1620,20,0.01235
8,Capital One,1504,0,0.00000
9,Synchrony Financial,1375,123,0.08945


Agora sim temos grandes companhias que tem um sério problema de tempo de resposta para as reclamações. Vou ordenar novamente por percentual respondido

In [15]:
pd_reclamacoes_top400.sort_values(by=['percentual_respondidas', 'qtd_reclamacoes'], ascending=[True, False]).head(20)

,company,qtd_reclamacoes,reclamacoes_respondidas,percentual_respondidas
0,Equifax,4195,0,0.00000
6,JPMorgan Chase & Co.,2586,0,0.00000
8,Capital One,1504,0,0.00000
14,Encore Capital Group,865,0,0.00000
15,Amex,740,0,0.00000
16,"Portfolio Recovery Associates, Inc.",680,0,0.00000
19,Discover,560,0,0.00000
20,"Empowerment Ventures, LLC",508,0,0.00000
23,TD Bank US Holding Company,457,0,0.00000
22,PNC Bank N.A.,466,2,0.00429


Fiquei curioso com as duas reclamações respondidas da PNC Bank N.A. (nome bem sugestivo... "Pau no..").  
Vou carregar a tabela inteira, filtrar por companhia e ver se encontro essas reclamações que os clientes descreveram a narrativa e a empresa respondeu 

In [16]:
display(pd.read_sql_query("""select complaint_id, company, date_received, product, sub_product, issue, sub_issue, consumer_complaint_narrative, company_public_response, submitted_via, timely_response
                            from consumer_complaints
                            where company like 'PNC%' 
                            and consumer_complaint_narrative is not NULL
                            and company_public_response is not NULL
                        """, conn))

,complaint_id,company,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,submitted_via,timely_response
0,1347585,PNC Bank N.A.,04/26/2015,Mortgage,Conventional adjustable mortgage (ARM),"Loan modification,collection,foreclosure",None,I have submitted a request for release of a XX...,Company chooses not to provide a public response,Web,Yes
1,1650258,PNC Bank N.A.,11/10/2015,Credit card,None,Other fee,None,please make sure you escalate to someone other...,Company chooses not to provide a public response,Web,Yes


Olha... as duas reclamações que a PNC resolveu responder publicamente foi justamente para dizer que "a empresa decidiu não responder publicamente".  
Em outras palavras, "Pau no..." no cliente.   
Eu já cancelaria a conta na hora.  
**Mas vamos fingir que eles ligam pra atendimento ao cliente e contrataram NÓS AQUI DA CONSULTORIA para melhorar a imagem deles através da IA.**   
Vou primeiramente coletar numa tabela todas as reclamações da firma sem resposta.

In [17]:
pd_reclamacoes_pnc = pd.read_sql_query("""select complaint_id, company, date_received, product, sub_product, issue, sub_issue, consumer_complaint_narrative, company_public_response, submitted_via, timely_response
                            from consumer_complaints
                            where company like 'PNC%' 
                            and consumer_complaint_narrative is not NULL
                            and company_public_response is NULL
                        """, conn)
display(pd_reclamacoes_pnc)

,complaint_id,company,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,submitted_via,timely_response
0,1290617,PNC Bank N.A.,03/19/2015,Consumer Loan,Vehicle loan,Managing the loan or lease,None,I have a car loan with PNC bank. In XXXX of 20...,None,Web,Yes
1,1291299,PNC Bank N.A.,03/19/2015,Bank account or service,Cashing a check without an account,Deposits and withdrawals,None,I have a contract with a guy whom has an accou...,None,Web,Yes
2,1291414,PNC Bank N.A.,03/19/2015,Bank account or service,Checking account,"Account opening, closing, or management",None,I signed up for a PNC account on XXXX XXXX 201...,None,Web,Yes
3,1301711,PNC Bank N.A.,03/26/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",None,PNC Mortgage forces me to pay {$7.00} fee each...,None,Web,Yes
4,1303393,PNC Bank N.A.,03/26/2015,Bank account or service,Checking account,"Account opening, closing, or management",None,I opened a new checking account [ XXXX ] with ...,None,Web,Yes
...,...,...,...,...,...,...,...,...,...,...,...
459,1701859,PNC Bank N.A.,12/17/2015,Consumer Loan,Installment loan,Problems when you are unable to pay,None,I intended to buy a home with a mortgage in 20...,None,Web,Yes
460,1775457,PNC Bank N.A.,02/05/2016,Money transfers,Domestic (US) money transfer,Other transaction issues,None,"My daughter set me up as a payee in PNC 's "" p...",None,Web,Yes
461,1381042,PNC Bank N.A.,05/18/2015,Bank account or service,Checking account,Problems caused by my funds being low,None,My bank statement reflects that on the XX/XX/X...,None,Web,Yes
462,1765721,PNC Bank N.A.,01/31/2016,Bank account or service,Other bank product/service,"Account opening, closing, or management",None,"On XXXX/XXXX/2015, PNC Bank accessed my XXXX p...",None,Web,Yes


Vou remover as chaves { e } do texto para não dar problema no Prompt Template. Para isso vou usar uma função apply para aplicar em toda coluna da reclamação do cliente

In [80]:
def remove_chaves(text):
    return text.replace('{', '').replace('}', '')

pd_reclamacoes_pnc['consumer_complaint_narrative'] = pd_reclamacoes_pnc['consumer_complaint_narrative'].apply(remove_chaves)
display(pd_reclamacoes_pnc)

,complaint_id,company,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,submitted_via,timely_response
0,1290617,PNC Bank N.A.,03/19/2015,Consumer Loan,Vehicle loan,Managing the loan or lease,None,I have a car loan with PNC bank. In XXXX of 20...,None,Web,Yes
1,1291299,PNC Bank N.A.,03/19/2015,Bank account or service,Cashing a check without an account,Deposits and withdrawals,None,I have a contract with a guy whom has an accou...,None,Web,Yes
2,1291414,PNC Bank N.A.,03/19/2015,Bank account or service,Checking account,"Account opening, closing, or management",None,I signed up for a PNC account on XXXX XXXX 201...,None,Web,Yes
3,1301711,PNC Bank N.A.,03/26/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",None,PNC Mortgage forces me to pay $7.00 fee each t...,None,Web,Yes
4,1303393,PNC Bank N.A.,03/26/2015,Bank account or service,Checking account,"Account opening, closing, or management",None,I opened a new checking account [ XXXX ] with ...,None,Web,Yes
...,...,...,...,...,...,...,...,...,...,...,...
459,1701859,PNC Bank N.A.,12/17/2015,Consumer Loan,Installment loan,Problems when you are unable to pay,None,I intended to buy a home with a mortgage in 20...,None,Web,Yes
460,1775457,PNC Bank N.A.,02/05/2016,Money transfers,Domestic (US) money transfer,Other transaction issues,None,"My daughter set me up as a payee in PNC 's "" p...",None,Web,Yes
461,1381042,PNC Bank N.A.,05/18/2015,Bank account or service,Checking account,Problems caused by my funds being low,None,My bank statement reflects that on the XX/XX/X...,None,Web,Yes
462,1765721,PNC Bank N.A.,01/31/2016,Bank account or service,Other bank product/service,"Account opening, closing, or management",None,"On XXXX/XXXX/2015, PNC Bank accessed my XXXX p...",None,Web,Yes


### Hora de botar a IA pra trabalhar

In [ ]:
import os
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

Pegarei uma amostra de 5 reclamações únicas aleatórias

In [ ]:
# Função sample coleta uma amostra aleatória de um DataFrame.
# Defini um random_state para toda vez que rodar, selecionar a mesma amostra. 
# O Replace = False garante que não haverá repetição de linhas na amostra.

df_sample = pd_reclamacoes_pnc.sample(5, random_state=42, replace=False)
display(df_sample)

,complaint_id,company,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,submitted_via,timely_response
9,1294663,PNC Bank N.A.,03/21/2015,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",None,PNC is servicing my mortgage. I filed for bank...,None,Web,Yes
30,1312581,PNC Bank N.A.,04/02/2015,Bank account or service,Checking account,Problems caused by my funds being low,None,I bank with PNC Bank. I have for 10 years. Rec...,None,Web,Yes
39,1346696,PNC Bank N.A.,04/24/2015,Bank account or service,Checking account,"Account opening, closing, or management",None,PNC bank is falsely submitting overdraft fees \n,None,Web,Yes
443,1800494,PNC Bank N.A.,02/23/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,None,I have a vehicle financed of which i paid a to...,None,Web,Yes
266,1576383,PNC Bank N.A.,09/23/2015,Mortgage,FHA mortgage,Settlement process and costs,None,"I am the XXXX for buyers of XXXX XXXX XXXX, XX...",None,Web,Yes


Configuraremos o modelo de IA para executar as tarefas do desafio.   
Precisamos carregar as chaves de acesso utilizando load_dotenv().  
**Não esqueça de criar um arquivo texto .env com as credenciais necessárias!!!**

In [ ]:
load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(temperature=0,
                 model_name="gpt-4o-mini")


Criando classe com as 5 funções que utilizará a IA para processamento das reclamações

In [96]:
class ComplainSolverIA():
    def __init__(self, llm_model):
        self.llm = llm_model

    def function_ia_translator(self, user_message):
        """Esta função tem o objetivo de traduzir o prompt do inglês para português""" 

        prompt = f"""Você é um especialista em inglês. Seu objetivo é traduzir o texto abaixo para o português. 
                            \n\n{user_message}"""
        
        prompt_template = PromptTemplate(input_variables=["user_message"],
                                        template=prompt)
        
        chain = prompt_template | self.llm | StrOutputParser()

        return chain.invoke(input={"user_message": user_message})


    def function_ia_summary(self, user_message):
        """Esta função tem o objetivo de sumarizar a reclamação do cliente""" 

        prompt = f"""Você é encerregado em resumir em até 100 caracteres o resumo da reclamação do usuário para 
                    a empresa para passar a reclamação para o departamento de avaliação de risco.  
                            \n\n{user_message}"""
        
        prompt_template = PromptTemplate(input_variables=["user_message"],
                                        template=prompt)
        
        chain = prompt_template | self.llm | StrOutputParser()

        return chain.invoke(input={"user_message": user_message})
    
    def function_analysis_sentiment(self, user_message):
        """Esta função tem o objetivo de avaliar o grau de sentimento da mensagem.""" 

        prompt = f"""Você fará o papel do usuário que está reclamando. Sua missão é analisar o conteúdo da reclamação
                    e classificar o sentimento predominante da mensagem. 
                    Reclamação do cliente:\n
                    {user_message}"""
        
        prompt_template = PromptTemplate(input_variables=["user_message"],
                                        template=prompt)
        
        chain = prompt_template | self.llm | StrOutputParser()

        return chain.invoke(input={"user_message": user_message})
    
    def function_risk_evaluator(self, user_message):
        """Esta função tem o objetivo de avaliar o grau de risco do cliente com relação a reclamação.""" 

        prompt = f"""Você é um funcionário de avaliação de risco de uma empresa do segmento financeiro.
                    Sua função é a partir da reclamação do usuário, classificar o grau de risco de deixar a empresa em ALTO, MEDIO e BAIXO.
                    Segue o contexto para se levar em consideração na avaliação do risco do cliente:
                    \n
                    Risco Alto: Palavras que ameaça diretamente a empresa com processo, advogado, linguagem 
                    agressiva ou tóxica. Ameaça deixar de utilizar os produtos da empresa.
                    Risco Médio: Cliente não ameaça diretamente a empresa mas utiliza algumas palavras agressivas 
                    demonstrando sinal de estresse mas não dá indícios de deixar de consumir algum produto da empresa.
                    Risco Baixo: Cliente faz a reclamação sem ameaças a empresa e demonstra insatisfação de forma educada.
                    \n
                    Repare o tom de sentimento da mensagem do cliente. Se ele apresentar sinais claros de desespero, frustração
                    ou estresse, o risco desse cliente é ALTO porque ele não agirá mais com o racional. 
                    \n
                    Dito isto, retorne em apenas e unicamente com a string "ALTO", "MEDIO" ou "BAIXO" a reclamação do cliente.
                    \n
                    Reclamação do cliente:\n
                    {user_message}"""
        
        prompt_template = PromptTemplate(input_variables=["user_message"],
                                        template=prompt)
        
        chain = prompt_template | self.llm | StrOutputParser()

        return chain.invoke(input={"user_message": user_message})
    
    def function_came_to_solve_everything(self, user_message):
        """Esta função tem o objetivo de avaliar o grau de risco do cliente com relação a reclamação.""" 

        prompt = f"""Você é um especialistas em relações públicas. Sua missão é gerar uma mensagem personalizada em até
                    200 caracteres para enviar ao cliente. A mensagem deve contemplar:
                    1 - O problema especificado
                    2 - A preocupação da empresa em resolvê-lo
                    3 - Pedido de desculpas APENAS e UNICAMENTE se a EMPRESA foi a CULPADA pelo problema.
                    Reclamação do cliente:\n
                    {user_message}
                    Saída esperada:
                    Mensagem para o Cliente:"""
        
        prompt_template = PromptTemplate(input_variables=["user_message"],
                                        template=prompt)
        
        chain = prompt_template | self.llm | StrOutputParser()

        return chain.invoke(input={"user_message": user_message})



Criando a função que gera essa tabela de reclamação

In [105]:
def complains_table(df_table, llm_model):
    """
        Esta função monta a tabela final com as reclamações dos clientes, resumo da reclamação, sentimento da reclamação,
        risco do cliente e mensagem de ação para o cliente.
    """
    complaint_id = df_table['complaint_id'].values
    complaint_date = df_table['date_received'].values
    company = df_table['company'].values
    reclamacao = []
    resumo_reclamacao = []
    sentimento_reclamacao = []
    risco_cliente = []
    mensagem_acao = []

    ia = ComplainSolverIA(llm_model)

    for _, row in df_table.iterrows():
        print(f"Processando reclamação {row['complaint_id']}")
        reclamacao.append(ia.function_ia_translator(row['consumer_complaint_narrative']))
        resumo_reclamacao.append(ia.function_ia_summary(row['consumer_complaint_narrative']))
        sentimento_reclamacao.append(ia.function_analysis_sentiment(row['consumer_complaint_narrative']))
        risco_cliente.append(ia.function_risk_evaluator(row['consumer_complaint_narrative']))
        mensagem_acao.append(ia.function_came_to_solve_everything(row['consumer_complaint_narrative']))

    return pd.DataFrame({'id_reclamacao': complaint_id,
                         'data_reclamacao': complaint_date,
                         'empresa': company,
                         'reclamacao_cliente': reclamacao,
                         'resumo_reclamacao_cliente': resumo_reclamacao,
                         'sentimento_reclamacao': sentimento_reclamacao,
                         'risco_cliente': risco_cliente,
                         'mensagem_acao': mensagem_acao})

Exibindo a tabela final

In [106]:
df_final = complains_table(df_sample, llm)
display(df_final)

Processando reclamação 1294663
Processando reclamação 1312581
Processando reclamação 1346696
Processando reclamação 1800494
Processando reclamação 1576383


,id_reclamacao,data_reclamacao,empresa,reclamacao_cliente,resumo_reclamacao_cliente,sentimento_reclamacao,risco_cliente,mensagem_acao
0,1294663,03/21/2015,PNC Bank N.A.,A PNC está administrando a minha hipoteca. Eu ...,"PNC atrasou venda pós-falência, gerando multas...","Após analisar o conteúdo da reclamação, o sent...",ALTO,Lamentamos profundamente os transtornos enfren...
1,1312581,04/02/2015,PNC Bank N.A.,Eu sou cliente do PNC Bank há 10 anos. Recente...,PNC desconta valores indevidos e cobra taxas e...,**Análise de Sentimento:**\n\nO sentimento pre...,ALTO,Mensagem para o Cliente:\n\nLamentamos os inco...
2,1346696,04/24/2015,PNC Bank N.A.,O banco PNC está cobrando taxas de descoberto ...,PNC Bank está cobrando taxas de overdraft inde...,**Sentimento Predominante:** Negativo\n\n**Aná...,MEDIO,"Prezado(a),\n\nPedimos desculpas pelos encargo..."
3,1800494,02/23/2016,PNC Bank N.A.,"Claro, aqui está a tradução do texto para o po...","Financiamento pago integralmente, cobrança ind...",A reclamação do cliente expressa predominantem...,ALTO,"Olá [Cliente], lamentamos pelo transtorno com ..."
4,1576383,09/23/2015,PNC Bank N.A.,Eu sou o XXXX para os compradores de XXXX XXXX...,"Atrasos na avaliação impedem fechamento, apesa...",**Sentimento predominante:** Negativo\n\n**Aná...,MEDIO,**Mensagem para o Cliente:**\n\nLamentamos pel...


Exportando para csv

In [108]:
df_final.to_csv('reclamacoes_final.csv', index=False, encoding='utf-8')